In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from auction.operations import create_auction_app, setup_auction_app, place_bid, close_auction
from store.operations import StoringPool
from utils import get_algod_client, wait_for_confirmation, get_balances, get_last_block_timestamp, get_app_global_state, get_app_local_state
from account import Account

In [2]:
dotenv.load_dotenv('.env')
print(os.environ.get('ALGOD_URL'))

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

http://localhost:4001


In [3]:
def optin_to_asset(asset_id, sender: Account):
    txn = transaction.AssetOptInTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=asset_id
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    wait_for_confirmation(client, signed_txn.get_txid())

In [4]:
creator = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
seller = Account.from_mnemonic(os.environ.get("SELLER_MN"))

print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [5]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

#asset_id = create_dummy_asset(creator, 1000000000000000000, 3, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [6]:
start_time = int(time()) + 10
end_time = start_time + 30
print(f"start_time", start_time)
reserve = 1_000_000  # 1 Algo
token_amount = 100
increment = 100_000  # 0.1 Algo

start_time 1640189360


In [7]:
#close_auction(client, 527, 123, creator)

In [8]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)

storeApp.create_app()
storeAppID = storeApp.app_id
# storeAppID = int(os.environ.get("STORE_APP_ID"))
# storeApp.app_id = storeAppID

bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))
bidder1 = Account.from_mnemonic(os.environ.get("BUYER1_MN"))
bidder2 = Account.from_mnemonic(os.environ.get("BUYER2_MN"))

if storeApp.is_opted_in(seller.get_address()) == False:
  storeApp.optin_app(seller)
if storeApp.is_opted_in(bidder.get_address()) == False:
  storeApp.optin_app(bidder)
if storeApp.is_opted_in(bidder1.get_address()) == False:
  storeApp.optin_app(bidder1)
if storeApp.is_opted_in(bidder2.get_address()) == False:
  storeApp.optin_app(bidder2)

#close_auction(client, 527, storeAppID, creator)

print(f"storeAppID", storeAppID)
print(b"storeApp address", get_application_address(storeAppID))


Waiting for confirmation...
Transaction 4S3RCPLJ4BGPQJITGQ5ARLU56AVZ2J452QZWQ55CVFXE7G3BZS5A confirmed in round 23734.
Store App ID: 635
Store App address: MOFULCP7ACXJ4UOYWNDTZXEX3RDABJJFVYB2P7MZKJUNEIGOJHYBZR6TJU
Waiting for confirmation...
Transaction ROQ2RJD3UM6G6LVR75GW35LIECK2HQ5MI7UQG4CTRJWSH7DQJQGA confirmed in round 23736.
Waiting for confirmation...
Transaction SGW2COCFCQQCMFE5PXHNOGS43PZI6TJQ4GEIK6KMU5YQO4RIWTIQ confirmed in round 23738.
Waiting for confirmation...
Transaction RGZHI6J4VMPGETQYQJMJNYA3QDB37GEJBNHDOBIJHE6AXPOETSPQ confirmed in round 23740.
Waiting for confirmation...
Transaction XSXJXKBNSLQGDXYQFO7A2T6MUU4PIDTTIGOLOUX4GOHIU2VYA2EA confirmed in round 23742.
Waiting for confirmation...
Transaction 6WT3S2YDS3SM5ZC3ETTGXZYMXFCPACYB67MAPLPQK4QPVSMYKJZA confirmed in round 23744.
storeAppID 635
b'storeApp address' MOFULCP7ACXJ4UOYWNDTZXEX3RDABJJFVYB2P7MZKJUNEIGOJHYBZR6TJU


"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [9]:

print(f"seller address: {seller.get_address()}")
appID = create_auction_app(
    client=client,
    sender=creator,
    seller=seller.get_address(),
    token_id=asset_id,
    token_amount=token_amount,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment,
    store_app_address=get_application_address(storeAppID)
)

#appID = 527
print(f"Auction App ID: {appID}")
print(f"Auction App Address: {get_application_address(appID)}")
print("Latest time:", get_app_global_state(client, appID))

seller address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM
Waiting for confirmation...
Transaction 5XZ6YQCJKVUAEU4HASW26Y5ZQR4EOZZIYIG4QKHGP3MG3TPZHR3A confirmed in round 23746.
Auction App ID: 641
Auction App Address: BN67VT3GVLNRS4GKKGDEZ6QPM6VVNOGGFKL6Q3K34TDKU2L7AWRHX77FKE
Latest time: {b'TWA': b"\xa0\x81}\xb3\xa7\x97\xdfk,>8\xc4\xcd;\xe9'\x0e-\xc4\xc1{lX\xbf\x92\xc5=\xac\xcb\xad\x94\xb4", b'B_ADDR': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', b'BA': 0, b'TKA': 100, b'DAA': b'0\n\xbcQ\xb5>`\xa8\x1f\xf8Y\xeaV\xb5\xfd\xb3\xa7q\x83\xed\xdb\x13\x1a\xa7\xd2,\x8f\xf0\xc6\xbcW\xd0', b'min_bid_inc': 100000, b'end': 1640189390, b'reserve_amount': 1000000, b'num_bids': 0, b'SAA': b'c\x8bE\x89\xff\x00\xae\x9eQ\xd8\xb3G<\xdc\x97\xdcF\x00\xa5%\xae\x03\xa7\xfd\x99Rh\xd2 \xceI\xf0', b'TK_ID': 499, b'S_ADDR': b'\xc7!\x1f\xac\x93\xa5\xd3\n\xc9\xf1S\x93\xda\x94q\xd1\xa2\xed\xb2rZx%\xc9\x0fOp\xdd\

Alice is setting up and funding token auction...

In [10]:
setup_auction_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_holder=seller,
    token_id=asset_id,
    asset_amount=token_amount
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

fund_app_txn: {'sender': 'GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI', 'fee': 1000, 'first_valid_round': 23746, 'last_valid_round': 24746, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'BN67VT3GVLNRS4GKKGDEZ6QPM6VVNOGGFKL6Q3K34TDKU2L7AWRHX77FKE', 'amt': 203000, 'close_remainder_to': None}
setup_txn: {'sender': 'GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI', 'fee': 1000, 'first_valid_round': 23746, 'last_valid_round': 24746, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 641, 'on_complete': <OnComplete.NoOpOC: 0>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': [b'setup'], 'accounts': None, 'foreign_apps': None, 'foreign_assets': [499], '

In [11]:
bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))
bidder1 = Account.from_mnemonic(os.environ.get("BUYER1_MN"))
bidder2 = Account.from_mnemonic(os.environ.get("BUYER2_MN"))

print(f"Bidder Address: {bidder.get_address()}")
print(f"Bidder1 Address: {bidder1.get_address()}")
print(f"Bidder2 Address: {bidder2.get_address()}")

Bidder Address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Bidder1 Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
Bidder2 Address: 742N2DLQ6NUUQXNUTNIJHYXYGL6BWG7C2XSQDHLFJN2NFYDVAYYUAQ36NQ


In [12]:
# _, last_round_time = get_last_block_timestamp(client)
# print(f"start_time", start_time)
# print(f"last_round_time", last_round_time)
# if last_round_time < start_time + 5:
#     sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 202000, 499: 100}


In [13]:
bidAmount = reserve
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

Carla wants to bid on token, her algo balance:  10005906815659  algos
Carla is placing bid for:  1000000  algos


In [14]:
_, lastRoundTime = get_last_block_timestamp(client)
print(f"lastRoundTime", lastRoundTime)
place_bid(client=client, app_id=appID, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

lastRoundTime 1640156970
Waiting for confirmation...
Transaction USGN3BISOTKHM6OVZAMKJER3TDGI2DD4GDOONSOJPRUBVIYEKMRA confirmed in round 23750.
Carla is opting into token with id: 499


In [15]:
optin_to_asset(asset_id, bidder)

bidAmount = reserve + 300_000
place_bid(client=client, app_id=appID, bidder=bidder1, bid_amount=bidAmount)

optin_to_asset(asset_id, bidder1)

bidAmount = reserve + 500_000
place_bid(client=client, app_id=appID, bidder=bidder2, bid_amount=bidAmount)

optin_to_asset(asset_id, bidder2)

Waiting for confirmation...
Transaction S27MFGMVPJTECAXAONLOUGFBPPMPDPKODKHIKZQCSGLIMM6BB6IQ confirmed in round 23752.
Waiting for confirmation...
Transaction H3KHT3FPAWUOTXV24JURDH2DWMJEWXYL2DIEXMJCOAEJX26AK4ZA confirmed in round 23754.
Waiting for confirmation...
Transaction BG4NRF5CWLBMTKX4X4M4YGQXZG6WL7HANEFTAELHQ5H56GXITLRA confirmed in round 23756.
Waiting for confirmation...
Transaction BNI6BCROGYYLM2QSA53L4EW43QCTM5NYENP2HMZC3TIPQGYJRC2Q confirmed in round 23758.
Waiting for confirmation...
Transaction 5H52F3VG3FGSQHD4O2NK6PPEFXCAL6ABAUKG2LM36AJYWZ7VCP6Q confirmed in round 23760.


Alice is closing out the auction....

In [16]:
# _, lastRoundTime = get_last_block_timestamp(client)
# if lastRoundTime < end_time + 5:
#     sleep(end_time + 5 - lastRoundTime)

close_auction(client, appID, storeAppID, creator, seller)

app_global_state {b'BA': 1500000, b'TKA': 100, b'S_ADDR': b'\xc7!\x1f\xac\x93\xa5\xd3\n\xc9\xf1S\x93\xda\x94q\xd1\xa2\xed\xb2rZx%\xc9\x0fOp\xdd\xef;\xcf\xce', b'B_ADDR': b'\xff4\xdd\rp\xf3iH]\xb4\x9bP\x93\xe2\xf82\xfc\x1b\x1b\xe2\xd5\xe5\x01\x9deKt\xd2\xe0u\x061', b'DAA': b'0\n\xbcQ\xb5>`\xa8\x1f\xf8Y\xeaV\xb5\xfd\xb3\xa7q\x83\xed\xdb\x13\x1a\xa7\xd2,\x8f\xf0\xc6\xbcW\xd0', b'SAA': b'c\x8bE\x89\xff\x00\xae\x9eQ\xd8\xb3G<\xdc\x97\xdcF\x00\xa5%\xae\x03\xa7\xfd\x99Rh\xd2 \xceI\xf0', b'TWA': b"\xa0\x81}\xb3\xa7\x97\xdfk,>8\xc4\xcd;\xe9'\x0e-\xc4\xc1{lX\xbf\x92\xc5=\xac\xcb\xad\x94\xb4", b'TK_ID': 499, b'num_bids': 3, b'start': 1640189360, b'reserve_amount': 1000000, b'min_bid_inc': 100000, b'end': 1640189390}
token_amount 100
store_buying_txn: {'sender': 'GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI', 'fee': 1000, 'first_valid_round': 23760, 'last_valid_round': 24760, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'gr

In [22]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)
print(f"storeAppID", storeAppID)
#assert actualAppBalances == expectedAppBalances

from utils import get_app_local_state
print(get_app_local_state(client, storeAppID, seller.get_address()))
print(get_app_local_state(client, storeAppID, bidder.get_address()))
print(get_app_local_state(client, storeAppID, bidder1.get_address()))
print(get_app_local_state(client, storeAppID, bidder2.get_address()))

The smart contract now holds the following: {0: 143647}
storeAppID 635
{b'TKSA': 100}
{}
{}
{b'TKBA': 100}


In [ ]:
#bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]
bidder1NftBalance = get_balances(client, bidder1.get_address())[asset_id]
#bidder2NftBalance = get_balances(client, bidder2.get_address())[asset_id]

print("Carla's NFT balance:", bidder1NftBalance, " for token ID: ", asset_id)

assert bidder1NftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, 591)
print(storeAppState)